#### Includes

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import altair as alt
from altair import datum
import pandas as pd
import time
import os.path as P
from pytorch_lightning.utilities.cloud_io import load as pl_load
import pandas as pd
import altair as alt
alt.data_transformers.disable_max_rows()
None

In [3]:
from sources import load_dataset, default_config, parse_config
from run_train import setup_model
from src.trainer import BinaryClassifierNet

In [4]:
from steered_cnn.utils.rotequivariance_toolbox import *
from steered_cnn.steered_conv.steerable_filters import radial_steerable_filter, plot_filter
from steered_cnn.steered_conv import SteerableKernelBase

## Initialization

In [5]:
RUN_ID = '8/dd8fc55431b5441e811d33871cd9aca6'
OLD = False

In [6]:
art_path = P.abspath(P.join('/home/gaby/mlflow/mlruns', RUN_ID, 'artifacts/'))
cfg = parse_config(art_path+'/cfg_extended.yaml')
cfg['script-arguments']['gpus']='0'

In [7]:
model = setup_model(cfg.model, old=OLD)
net = BinaryClassifierNet(model=model)
state_dict = pl_load(art_path+'/best-val-roc.ckpt'.format(25), map_location='cuda:0')['state_dict']
net.load_state_dict(state_dict)
net.eval().cuda()
None

## Weight Value Analysis

In [8]:
base = model.base
weights = [_ for _ in model.parameters() if (_.ndim==3 and _.shape[-1]==len(base.kernels_label))]

In [9]:
W = [_ for _ in model.parameters() if (_.ndim==3 and _.shape[-1]==len(base.kernels_label))]

In [10]:
base.plot_weights_hist(W, binspace='log', wrange=(1e-5,1), bins=1000)

alt.Chart(...)

In [33]:
base.plot_weights_dist(weights, scale_type='log', wrange=(1e-4,1))

alt.FacetChart(...)

In [35]:
base.plot_weights_dist(base.create_weights(100,500),  scale_type='log', wrange=(1e-3,1e-1))

alt.FacetChart(...)

## Gradient Analysis

In [13]:
trainD, validD, testD = load_dataset(cfg)

/home/gaby/.conda/envs/nnet/lib/python3.7/site-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [14]:
net.train()
net.zero_grad()
for batch in trainD:
    batch_cuda = {k: v.cuda() for k, v in batch.items()}
    loss = net.training_step(batch_cuda, 0)
    loss.backward()
    break

In [30]:
weights_grad = [w.grad for w in weights]
base.plot_weights_dist(weights_grad, scale_type='log', wrange=(1e-10, 1e-4))

alt.FacetChart(...)

In [16]:
base.plot_weights_hist(weights_grad, complex='norm', wrange=(1e-11, 1e-2), binspace='log')

alt.Chart(...)